# Data Analyst Code Submission - Bhavya Passi

Importing required libraries and using Pandas to clean and structure the data.

In [58]:
import gzip
import shutil
import os
import json
import pandas as pd
from ast import literal_eval
from sqlalchemy import create_engine
from datetime import datetime

Reading all the required json files

In [59]:
receipts = pd.read_json('receipts.json',lines=True)
brands = pd.read_json('brands.json',lines=True)
users = pd.read_json('users.json',lines=True)

## First: Review Existing Unstructured Data and Diagram a New Structured Relational Data Model

#### Receipts

In [60]:
receipts.head()

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,{'$oid': '5ff1e1eb0a720f0523000575'},500.0,"Receipt number 2 completed, bonus point schedu...",{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687536000},{'$date': 1609687531000},500.0,{'$date': 1609632000000},5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.0,5ff1e1eacfcf6c399c274ae6
1,{'$oid': '5ff1e1bb0a720f052300056b'},150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,{'$date': 1609601083000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.0,5ff1e194b6a9d73a3a9f1052
2,{'$oid': '5ff1e1f10a720f052300057a'},5.0,All-receipts receipt bonus,{'$date': 1609687537000},{'$date': 1609687537000},NaN,{'$date': 1609687542000},NaN,5.0,{'$date': 1609632000000},1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.0,5ff1e1f1cfcf6c399c274b0b
3,{'$oid': '5ff1e1ee0a7214ada100056f'},5.0,All-receipts receipt bonus,{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687539000},{'$date': 1609687534000},5.0,{'$date': 1609632000000},4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.0,5ff1e1eacfcf6c399c274ae6
4,{'$oid': '5ff1e1d20a7214ada1000561'},5.0,All-receipts receipt bonus,{'$date': 1609687506000},{'$date': 1609687506000},{'$date': 1609687511000},{'$date': 1609687511000},{'$date': 1609687506000},5.0,{'$date': 1609601106000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.0,5ff1e194b6a9d73a3a9f1052


Using explode function on "rewardsReceiptItemList", I will first break the string into an array. Post this, I will use the literal_eval function from ast package.
Finally, I will be splitting each dictionary and get all the columns from it using the json_normalize function and then join all these columns back into the Receipts data frame.

In [61]:
# Receipts 
receipts = receipts.explode('rewardsReceiptItemList')
receipts.reset_index(inplace=True)
receipts = receipts.fillna({'rewardsReceiptItemList':'{}'})
receipts['rewardsReceiptItemList'] = receipts['rewardsReceiptItemList'].apply(lambda x:str(x))
receipts['rewardsReceiptItemList'] = receipts['rewardsReceiptItemList'].apply(literal_eval)
receipts_norm = pd.json_normalize(receipts['rewardsReceiptItemList'],errors='ignore',record_prefix='rewardsReceiptItemList')\
.add_prefix('rewardsReceiptItemList.')
receipts_updated = pd.merge(receipts, receipts_norm, left_index=True, right_index=True, how='outer')
receipts_updated.head()

,index,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,...,rewardsReceiptItemList.itemNumber,rewardsReceiptItemList.originalMetaBriteQuantityPurchased,rewardsReceiptItemList.pointsEarned,rewardsReceiptItemList.targetPrice,rewardsReceiptItemList.competitiveProduct,rewardsReceiptItemList.originalFinalPrice,rewardsReceiptItemList.originalMetaBriteItemPrice,rewardsReceiptItemList.deleted,rewardsReceiptItemList.priceAfterCoupon,rewardsReceiptItemList.metabriteCampaignId
0,0,{'$oid': '5ff1e1eb0a720f0523000575'},500.0,"Receipt number 2 completed, bonus point schedu...",{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687536000},{'$date': 1609687531000},500.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,{'$oid': '5ff1e1bb0a720f052300056b'},150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,{'$oid': '5ff1e1bb0a720f052300056b'},150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,{'$oid': '5ff1e1f10a720f052300057a'},5.0,All-receipts receipt bonus,{'$date': 1609687537000},{'$date': 1609687537000},NaN,{'$date': 1609687542000},NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3,{'$oid': '5ff1e1ee0a7214ada100056f'},5.0,All-receipts receipt bonus,{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687539000},{'$date': 1609687534000},5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Formatting fields that have dictionaries

In [62]:
set().union(*(d.keys() for d in receipts_updated['_id']))
set().union(*(d.keys() for d in receipts_updated['createDate']))
set().union(*(d.keys() for d in receipts_updated['dateScanned']))
set().union(*(d.keys() for d in receipts_updated.dropna(subset=['finishedDate'])['finishedDate']))
set().union(*(d.keys() for d in receipts_updated.dropna(subset=['modifyDate'])['modifyDate']))
set().union(*(d.keys() for d in receipts_updated.dropna(subset=['pointsAwardedDate'])['pointsAwardedDate']))
set().union(*(d.keys() for d in receipts_updated.dropna(subset=['purchaseDate'])['purchaseDate']))

{'$date'}

Converting date column into datetime format by using function

In [63]:
def date_converter(x):
  try:
    return(datetime.utcfromtimestamp(int(x['$date'])/1000).strftime('%Y-%m-%d %H:%M:%S'))
  except TypeError:
    return(None)

In [64]:
receipts_updated['_id'] = receipts_updated['_id'].apply(lambda x: x['$oid'])
receipts_updated['createDate'] = receipts_updated['createDate'].apply(lambda x: date_converter(x))
receipts_updated['dateScanned'] = receipts_updated['dateScanned'].apply(lambda x: date_converter(x))
receipts_updated['finishedDate'] = receipts_updated['finishedDate'].apply(lambda x: date_converter(x))
receipts_updated['modifyDate'] = receipts_updated['modifyDate'].apply(lambda x: date_converter(x))
receipts_updated['pointsAwardedDate'] = receipts_updated['pointsAwardedDate'].apply(lambda x: date_converter(x))
receipts_updated['purchaseDate'] = receipts_updated['purchaseDate'].apply(lambda x: date_converter(x))

In [65]:
receipts_updated.head()

,index,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,...,rewardsReceiptItemList.itemNumber,rewardsReceiptItemList.originalMetaBriteQuantityPurchased,rewardsReceiptItemList.pointsEarned,rewardsReceiptItemList.targetPrice,rewardsReceiptItemList.competitiveProduct,rewardsReceiptItemList.originalFinalPrice,rewardsReceiptItemList.originalMetaBriteItemPrice,rewardsReceiptItemList.deleted,rewardsReceiptItemList.priceAfterCoupon,rewardsReceiptItemList.metabriteCampaignId
0,0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:36,2021-01-03 15:25:31,500.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:48,2021-01-03 15:24:43,150.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:48,2021-01-03 15:24:43,150.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,2021-01-03 15:25:37,2021-01-03 15:25:37,None,2021-01-03 15:25:42,None,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3,5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:39,2021-01-03 15:25:34,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Users 

In [66]:
users.head()

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
1,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
2,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
3,{'$oid': '5ff1e1eacfcf6c399c274ae6'},True,{'$date': 1609687530554},{'$date': 1609687530597},consumer,Email,WI
4,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI


Using the same function we created earlier to convert date into datetime format

In [67]:
set().union(*(d.keys() for d in users['_id']))
set().union(*(d.keys() for d in users['createdDate']))
set().union(*(d.keys() for d in users.dropna(subset=['lastLogin'])['lastLogin']))
users['_id'] = users['_id'].apply(lambda x: x['$oid'])
users['createdDate'] = users['createdDate'].apply(lambda x: date_converter(x))
users['lastLogin'] = users['lastLogin'].apply(lambda x: date_converter(x))
users.head()

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI
1,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI
2,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI
3,5ff1e1eacfcf6c399c274ae6,True,2021-01-03 15:25:30,2021-01-03 15:25:30,consumer,Email,WI
4,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI


#### Brands

In [68]:
brands.head()

,_id,barcode,category,categoryCode,cpg,name,topBrand,brandCode
0,{'$oid': '601ac115be37ce2ead437551'},511111019862,Baking,BAKING,"{'$id': {'$oid': '601ac114be37ce2ead437550'}, ...",test brand @1612366101024,0.0,NaN
1,{'$oid': '601c5460be37ce2ead43755f'},511111519928,Beverages,BEVERAGES,"{'$id': {'$oid': '5332f5fbe4b03c9a25efd0ba'}, ...",Starbucks,0.0,STARBUCKS
2,{'$oid': '601ac142be37ce2ead43755d'},511111819905,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176
3,{'$oid': '601ac142be37ce2ead43755a'},511111519874,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051
4,{'$oid': '601ac142be37ce2ead43755e'},511111319917,Candy & Sweets,CANDY_AND_SWEETS,"{'$id': {'$oid': '5332fa12e4b03c9a25efd1e7'}, ...",test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827


In [69]:
set().union(*(d.keys() for d in brands['cpg']))
brands_norm = pd.json_normalize(brands['cpg'])
brands_norm = brands_norm.add_prefix('cpg.')
brands_final = pd.merge(brands, brands_norm, left_index=True, right_index=True, how='outer')
brands_final['_id'] = brands_final['_id'].apply(lambda x: x['$oid'])
brands_final.head()

,_id,barcode,category,categoryCode,cpg,name,topBrand,brandCode,cpg.$ref,cpg.$id.$oid
0,601ac115be37ce2ead437551,511111019862,Baking,BAKING,"{'$id': {'$oid': '601ac114be37ce2ead437550'}, ...",test brand @1612366101024,0.0,NaN,Cogs,601ac114be37ce2ead437550
1,601c5460be37ce2ead43755f,511111519928,Beverages,BEVERAGES,"{'$id': {'$oid': '5332f5fbe4b03c9a25efd0ba'}, ...",Starbucks,0.0,STARBUCKS,Cogs,5332f5fbe4b03c9a25efd0ba
2,601ac142be37ce2ead43755d,511111819905,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176,Cogs,601ac142be37ce2ead437559
3,601ac142be37ce2ead43755a,511111519874,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051,Cogs,601ac142be37ce2ead437559
4,601ac142be37ce2ead43755e,511111319917,Candy & Sweets,CANDY_AND_SWEETS,"{'$id': {'$oid': '5332fa12e4b03c9a25efd1e7'}, ...",test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827,Cogs,5332fa12e4b03c9a25efd1e7


In [70]:
brands_final['brandCode']

0                                 NaN
1                           STARBUCKS
2       TEST BRANDCODE @1612366146176
3       TEST BRANDCODE @1612366146051
4       TEST BRANDCODE @1612366146827
                    ...              
1162                              NaN
1163               DIPPIN DOTS CEREAL
1164    TEST BRANDCODE @1598639215217
1165                LIPTON TEA Leaves
1166    TEST BRANDCODE @1613158231644
Name: brandCode, Length: 1167, dtype: object

## Second: Write a query that directly answers a predetermined question from a business stakeholder.

In [71]:
engine = create_engine('sqlite://', echo=False)
users.drop_duplicates(subset=['_id']).to_sql('users',con=engine)
receipts_updated.columns = receipts_updated.columns.str.replace(".", "_")
receipts_updated.drop(columns=['rewardsReceiptItemList'],axis=1).to_sql('receipts',con=engine)
brands_final.drop_duplicates(subset=['brandCode']).drop(columns=['cpg'],axis=1).to_sql('brands',con=engine)

/var/folders/6d/2l2gpsmd14z3ysqzfkwrx6m80000gn/T/ipykernel_34577/132017943.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  receipts_updated.columns = receipts_updated.columns.str.replace(".", "_")


In [72]:
engine.execute("SELECT r.rewardsReceiptItemList_brandCode,SUM(r.totalSpent) as summedTotalSpent from receipts r").fetchall()

[(None, 9506137.270000162)]

### When considering total number of items purchased from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?

In [73]:
engine.execute("SELECT r.rewardsReceiptStatus,sum(purchasedItemCount)  from receipts r group by r.rewardsReceiptStatus").fetchall()

[('FINISHED', 1364998.0),
 ('FLAGGED', 294816.0),
 ('PENDING', None),
 ('REJECTED', 740.0),
 ('SUBMITTED', None)]

### When considering average spend from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?

In [74]:
engine.execute("SELECT r.rewardsReceiptStatus,avg(totalSpent)  from receipts r group by r.rewardsReceiptStatus").fetchall()

[('FINISHED', 1244.372934121628),
 ('FLAGGED', 2635.5702469136068),
 ('PENDING', 28.03244897959184),
 ('REJECTED', 19.544970059880253),
 ('SUBMITTED', None)]

## Third: Evaluate Data Quality Issues in the Data Provided

### Checking number of users who have transacted with us and number of users in the table 

In [75]:
engine.execute("SELECT count(distinct userId)  from receipts r ").fetchall()

[(258,)]

In [76]:
engine.execute("SELECT count(distinct _id)  from users r ").fetchall()

[(212,)]

We have observed that certain users have transacted with us and have details present in the receipts table however their PII information is not present in the users table. While this could be possible as certain users have transacted with us way before the data was captured. To remove any duplicacies, we can remove these users or capture their information when they next transact with us. Incase we find the details match existing users (duplicates) we can map them to remove any data discrepancy. 

### Comparing unique brand codes with receipts

In [77]:
len(set(receipts_updated.dropna(subset=['rewardsReceiptItemList_brandCode'])['rewardsReceiptItemList_brandCode']))

227

In [78]:
len(set(brands_final.dropna(subset=['brandCode'])['brandCode']))

897

Comparing unique brand codes with receipts

In [79]:
receipts_dropped = receipts_updated.dropna(subset=['rewardsReceiptItemList_brandCode'])
receipts_dropped['rewardsReceiptItemList_brandCode'] = receipts_dropped['rewardsReceiptItemList_brandCode'].apply(lambda x:str(x))

/var/folders/6d/2l2gpsmd14z3ysqzfkwrx6m80000gn/T/ipykernel_34577/2016687530.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  receipts_dropped['rewardsReceiptItemList_brandCode'] = receipts_dropped['rewardsReceiptItemList_brandCode'].apply(lambda x:str(x))


In [80]:
brands_final['barcode'] = brands_final['barcode'].apply(lambda x: str(x))
brands_final['brandCode'] = brands_final['brandCode'].apply(lambda x: str(x))
receipts_with_brands = pd.merge(receipts_updated,brands_final,left_on='rewardsReceiptItemList_brandCode',right_on='brandCode')

In [81]:
set(receipts_dropped[~(receipts_dropped['rewardsReceiptItemList_brandCode'].isin(brands['brandCode']))]['rewardsReceiptItemList_brandCode'])

{'7UP',
 'ADVIL',
 'AMERICAN BEAUTY',
 'ARROWHEAD',
 'AZTECA',
 'BANZA',
 'BEAR CREEK COUNTRY KITCHENS',
 'BEN AND JERRYS',
 'BETTY CROCKER',
 'BIC',
 'BIGELOW',
 'BLUE DIAMOND',
 "BOAR'S HEAD",
 'BORDEN',
 'BOTA BOX',
 'BRAND',
 "BRASWELL'S",
 'BUNNY',
 "BUSH'S BEST",
 'C&H',
 'CADBURY',
 'CAL-ORGANIC FARMS',
 'CALIFIA FARMS',
 "CAMPBELL'S",
 'CARAMELLO',
 'CHEERIOS',
 'CHEESE',
 'CHEEZ-IT',
 'CHEX',
 'CHICKEN OF THE SEA',
 'CHIQUITA',
 'CINNAMON TOAST CRUNCH',
 'COKE',
 'COLEMAN NATURAL',
 "CONNIE'S PIZZA",
 'CREST 3D WHITE',
 'CRISPIX',
 'DANNON',
 'DARE',
 'DELI',
 'DIET COKE',
 'DIGIORNO',
 'DOLE',
 'DR PEPPER',
 'EDWARDS',
 "EGGLAND'S BEST",
 'EGGO',
 'EL MONTEREY',
 'ENERGIZER MAX',
 'ESSENTIAL EVERYDAY',
 'FAGE',
 "FAMOUS DAVE'S",
 "FLORIDA'S NATURAL",
 'FOLGERS',
 'FORTUNE YAKISOBA',
 'FRANZ',
 "FRENCH'S",
 'FRESH EXPRESS',
 'FRESH STEP',
 'FRONTERA',
 'GALLO FAMILY VINEYARDS',
 'GENERAL MILLS',
 'GERBER',
 'GERM-X',
 'GREEN GIANT',
 'GRIMMWAY FARMS',
 'HANOVER',
 'HARVEST SNA

Above listed are all the brand code but these codes are missing in the brands table.
Brands available in the receipt table are not available in the brand table - While logically this shouldn't be the case. While there might be multiple reasons this is possible, based on our understanding we believe that the brand's table is not updated with the latest brand codes while receipts are getting updated with recent brand id. 